<a href="https://colab.research.google.com/github/aran3004/Transformers/blob/main/BERT_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installs and Imports

In [3]:
pip install torch transformers requests beautifulsoup4 pandas numpy

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np

# Instantiate Model

In [3]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

# Encode and Calculate Sentiment

In [4]:
tokens =  tokenizer.encode('This is rubbish, I dont like it', return_tensors='pt') # here pt refers to pytorch

In [5]:
tokens

tensor([[  101, 10372, 10127, 13462, 75086, 13457,   117,   151, 11930, 11531,
         10197,   102]])

In [6]:
tokenizer.decode(tokens[0])

'[CLS] this is rubbish, i dont like it [SEP]'

In [7]:
result = model(tokens)
result

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.4801,  2.3259,  0.8947, -1.8496, -3.0202]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [8]:
int(torch.argmax(result.logits)) + 1

1

# Collect Reviews

In [20]:
r =  requests.get("https://www.amazon.co.uk/SPACE-GOODS-Chocolate-Superfood-Alternative/dp/B0B51W32W4?pd_rd_w=9nMVi&content-id=amzn1.sym.4bb95862-c4dc-4355-a3e7-a1e69e03b5af&pf_rd_p=4bb95862-c4dc-4355-a3e7-a1e69e03b5af&pf_rd_r=QAG8VW1KFQY177E1V4A8&pd_rd_wg=CuANq&pd_rd_r=51b0302b-d024-40d3-b94a-99e92781fd6d&pd_rd_i=B0B51W32W4&ref_=pd_bap_d_grid_rp_0_1_ec_pd_hp_d_atf_rp_2_i&th=1")
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*review-text-content.*')
results = soup.find_all('div', {'class': regex})
reviews = [result.text for result in results]

In [27]:
reviews[0]

'\nI gave in to the hype on social media marketing and bought this on an offer about a year or so ago. Ever since I have repeatedly bought more bags of this and can‚Äôt find a better alternative out there. As a coffee drinker (2-3 a day +) I have actually I only need one of these a day to keep me awake and energy levels maintained. I have done extensive research myself into the benefits of nootropics and mushrooms and they seem to be great for your health and brain function. I also love the taste, tastes like a proper, strong hot choclate. The only issue is it is very sweet which may be too much for some people and can get a lot of drinking every day. The price is also very high for what you get as the bag is pretty small and as the recommended portion size is a serving spoon you get through the bag very quickly- I only try to purchase this when there is a  discount/deal on. I am keen to try out the other coffee and vanilla flavours and see if these are any different but really enjoy h

# Load Reviews into DataFrame and Score

In [28]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [29]:
df.head()

,review
0,\nI gave in to the hype on social media market...
1,"\nI still have my coffee in the morning, but n..."
2,"\nI was sceptical about this stuff, but it hon..."
3,\nHear about this product and was greatly inte...
4,\nIm not usually a person into suppliments or ...


In [30]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits)) + 1

In [33]:
sentiment_score(df['review'].iloc[5])

1

In [34]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512])) # up to 512 tokens for NLP pipeline

In [35]:
df

,review,sentiment
0,\nI gave in to the hype on social media market...,5
1,"\nI still have my coffee in the morning, but n...",4
2,"\nI was sceptical about this stuff, but it hon...",4
3,\nHear about this product and was greatly inte...,2
4,\nIm not usually a person into suppliments or ...,4
5,\nThis is just an expensive chocolate caffeine...,1
6,\nIm on a pescatarian keto diet and miss my mo...,5
7,"\nGestern bekommen, heute erstmal probiert - s...",5
8,\nMit kalter Milch schmeckts mir am besten.\n,5
9,\nHab diesen ‚ÄûPilzkaffee‚Äú einfach gekauft ...,5
